In [1]:
!pip install numpy pandas statsmodels scikit-learn prophet

In [2]:

import os
import matplotlib.pyplot as plt

os.makedirs("figs", exist_ok=True)

def plot_por_contaminante(target_col, train, test, forecast, y_true, y_pred_prophet, y_pred_base, h, modelo_prophet):
    safe = target_col.replace(".", "_")

    # Ventana "histórica" para contexto en la gráfica 1 (detecta si tus datos son horarios o diarios)
    # Si es horario, muestra última semana (~7*24). Si es diario, última semana (~7).
    try:
        freq = pd.infer_freq(train["ds"])
    except Exception:
        freq = None
    if freq and ("H" in freq or "T" in freq or "min" in str(freq).lower()):
        n_back = min(len(train), 7*24)
        etiqueta_hist = "Train (últimos 7 días)"
    else:
        n_back = min(len(train), 7)
        etiqueta_hist = "Train (últimos 7)"

    # 1) Serie temporal: Observado (train reciente + test) vs Prophet vs Persistencia
    fig, ax = plt.subplots(figsize=(12, 4))
    if n_back > 0:
        ax.plot(train["ds"].iloc[-n_back:], train["y"].iloc[-n_back:], label=etiqueta_hist, alpha=0.6)
    ax.plot(test["ds"], y_true, label="Test (observado)", linewidth=1.8)
    ax.plot(test["ds"], y_pred_prophet, "--", label="Prophet (ŷ)", linewidth=1.5)
    ax.plot(test["ds"], y_pred_base, ":", label="Persistencia (y[t-1])", linewidth=1.5)
    ax.set_title(f"{target_col} • Observado vs Predicho (h={h})")
    ax.set_xlabel("Fecha")
    ax.set_ylabel(target_col)
    ax.grid(alpha=0.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(f"figs/{safe}_obs_pred.png", dpi=150)
    plt.close(fig)

    # 2) Residuos de Prophet en la ventana de prueba
    res = y_true - y_pred_prophet
    fig, ax = plt.subplots(figsize=(12, 3.5))
    ax.plot(test["ds"], res, linewidth=1.3)
    ax.axhline(0, color="k", lw=1)
    ax.set_title(f"{target_col} • Residuos (y - ŷ) en prueba")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Residuo")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(f"figs/{safe}_residuos.png", dpi=150)
    plt.close(fig)

    # 3) Dispersión Observado vs Predicho (calidad del ajuste)
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(y_true, y_pred_prophet, s=14, alpha=0.85)
    lo = float(np.nanmin([y_true.min(), y_pred_prophet.min()]))
    hi = float(np.nanmax([y_true.max(), y_pred_prophet.max()]))
    ax.plot([lo, hi], [lo, hi], "--", lw=1, color="k")
    ax.set_title(f"{target_col} • Observado vs ŷ (Prophet)")
    ax.set_xlabel("Observado")
    ax.set_ylabel("Predicho")
    ax.grid(alpha=0.3)
    fig.tight_layout()
    fig.savefig(f"figs/{safe}_scatter.png", dpi=150)
    plt.close(fig)

    # 4) Componentes del modelo (tendencias/estacionalidades/regresores)
    comp_fig = modelo_prophet.plot_components(forecast)
    comp_fig.set_size_inches(12, 8)
    comp_fig.tight_layout()
    comp_fig.savefig(f"figs/{safe}_componentes.png", dpi=150)
    plt.close(comp_fig)


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv("/suroesteFInal.csv")


df["ds"] = None
if "datetime" in df.columns:
    df["ds"] = pd.to_datetime(df["datetime"], errors="coerce")
elif "Fecha.y.hora" in df.columns:
    df["ds"] = pd.to_datetime(df["Fecha.y.hora"], errors="coerce")
else:
    raise ValueError("No encuentro datetime")


targets = ["PM2.5.ug.m3", "O3.ppb", "NO2.ppb"]
meteo_cols_all = ["TOUT.degC","RH..","WSR.KMPH","PRS.mmhg","SR.KW.m2","RAINF.mm.hr"]
meteo_cols = [c for c in meteo_cols_all if c in df.columns]  # filtra solo las que existan

for c in meteo_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

os.makedirs("figs", exist_ok=True)

def plot_por_contaminante(target_col, train, test, forecast, y_true, y_pred_prophet, y_pred_base, h, modelo_prophet):
    """
    Genera y guarda 4 figuras por contaminante:
    1) Serie Observado vs Prophet vs Persistencia
    2) Residuos en prueba
    3) Dispersión Observado vs Predicho
    4) Componentes de Prophet
    """
    safe = target_col.replace(".", "_")

    try:
        freq = pd.infer_freq(train["ds"])
    except Exception:
        freq = None
    if freq and ("H" in str(freq) or "T" in str(freq) or "min" in str(freq).lower()):
        n_back = min(len(train), 7*24)   # datos horarios: 7 días
        etiqueta_hist = "Train (últimos 7 días)"
    else:
        n_back = min(len(train), 7)      # datos diarios: 7 días
        etiqueta_hist = "Train (últimos 7)"

    # 1) Observado vs Predicho vs Persistencia (ventana de prueba)
    fig, ax = plt.subplots(figsize=(12, 4))
    if n_back > 0:
        ax.plot(train["ds"].iloc[-n_back:], train["y"].iloc[-n_back:], label=etiqueta_hist, alpha=0.6)
    ax.plot(test["ds"], y_true, label="Test (observado)", linewidth=1.8)
    ax.plot(test["ds"], y_pred_prophet, "--", label="Prophet (ŷ)", linewidth=1.5)
    ax.plot(test["ds"], y_pred_base, ":", label="Persistencia (y[t-1])", linewidth=1.5)
    ax.set_title(f"{target_col} • Observado vs Predicho (h={h})")
    ax.set_xlabel("Fecha"); ax.set_ylabel(target_col)
    ax.grid(alpha=0.3); ax.legend()
    fig.tight_layout(); fig.savefig(f"figs/{safe}_obs_pred.png", dpi=150); plt.close(fig)

    # 2) Residuos (y - ŷ) en prueba
    res = y_true - y_pred_prophet
    fig, ax = plt.subplots(figsize=(12, 3.5))
    ax.plot(test["ds"], res, linewidth=1.3)
    ax.axhline(0, color="k", lw=1)
    ax.set_title(f"{target_col} • Residuos (y - ŷ) en prueba")
    ax.set_xlabel("Fecha"); ax.set_ylabel("Residuo")
    ax.grid(alpha=0.3)
    fig.tight_layout(); fig.savefig(f"figs/{safe}_residuos.png", dpi=150); plt.close(fig)

    # 3) Dispersión Observado vs Predicho
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(y_true, y_pred_prophet, s=14, alpha=0.85)
    lo = float(np.nanmin([np.nanmin(y_true), np.nanmin(y_pred_prophet)]))
    hi = float(np.nanmax([np.nanmax(y_true), np.nanmax(y_pred_prophet)]))
    ax.plot([lo, hi], [lo, hi], "--", lw=1, color="k")
    ax.set_title(f"{target_col} • Observado vs ŷ (Prophet)")
    ax.set_xlabel("Observado"); ax.set_ylabel("Predicho")
    ax.grid(alpha=0.3)
    fig.tight_layout(); fig.savefig(f"figs/{safe}_scatter.png", dpi=150); plt.close(fig)

    # 4) Componentes del modelo (tendencias/estacionalidades/regresores)
    comp_fig = modelo_prophet.plot_components(forecast)
    comp_fig.set_size_inches(12, 8)
    comp_fig.tight_layout()
    comp_fig.savefig(f"figs/{safe}_componentes.png", dpi=150)
    plt.close(comp_fig)

all_results = []

lags = [1, 24, 168]  # 1 hora, 1 día, 1 semana (si tus datos son horarios)

for target_col in targets:
    df_lag = df.copy()
    # Variable objetivo a numérico
    df_lag["y"] = pd.to_numeric(df_lag[target_col], errors="coerce")

    # Crear lags de y
    for lag in lags:
        df_lag[f"y_lag{lag}"] = df_lag["y"].shift(lag)

    # Limpieza básica
    subset_cols = ["ds","y"] + meteo_cols + [f"y_lag{lag}" for lag in lags]
    df_lag = df_lag.dropna(subset=subset_cols).sort_values("ds").reset_index(drop=True)

    # Split: última h para prueba
    h = 48
    if len(df_lag) <= h:
        raise ValueError(f"No hay suficientes datos para {target_col}: {len(df_lag)} <= h={h}")
    train = df_lag.iloc[:-h]
    test  = df_lag.iloc[-h:]

    # Prophet
    m = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=False)
    # Estacionalidades para sub-diario: period en días (1 día, 7 días)
    m.add_seasonality(name="daily", period=1, fourier_order=6)
    m.add_seasonality(name="weekly", period=7, fourier_order=4)

    # Agregar regresores (meteo + lags)
    for col in meteo_cols + [f"y_lag{lag}" for lag in lags]:
        m.add_regressor(col)

    # Entrenar
    train_cols = ["ds","y"] + meteo_cols + [f"y_lag{lag}" for lag in lags]
    m.fit(train[train_cols])

    # Forecast sobre la ventana de prueba (usando mismos regresores)
    future = test[["ds"] + meteo_cols + [f"y_lag{lag}" for lag in lags]].copy()
    forecast = m.predict(future)

    # Evaluación Prophet
    y_true = test["y"].to_numpy()
    y_pred_prophet = forecast["yhat"].to_numpy()
    rmse_prophet = float(np.sqrt(mean_squared_error(y_true, y_pred_prophet)))
    mae_prophet  = float(mean_absolute_error(y_true, y_pred_prophet))

    # Baseline (persistencia y[t-1])
    y_pred_base = test["y_lag1"].to_numpy()
    rmse_base = float(np.sqrt(mean_squared_error(y_true, y_pred_base)))
    mae_base  = float(mean_absolute_error(y_true, y_pred_base))

    # Diferencias y mejoras porcentuales
    delta_rmse = rmse_base - rmse_prophet
    delta_mae  = mae_base - mae_prophet
    perc_rmse  = 100 * delta_rmse / rmse_base if rmse_base != 0 else np.nan
    perc_mae   = 100 * delta_mae  / mae_base if mae_base != 0 else np.nan

    # Guardar resultados extendidos
    all_results.append({
        "Variable": target_col,
        "RMSE_Prophet": rmse_prophet,
        "MAE_Prophet": mae_prophet,
        "RMSE_Baseline": rmse_base,
        "MAE_Baseline": mae_base,
        "ΔRMSE": delta_rmse,
        "ΔMAE": delta_mae,
        "Mejora_RMSE_%": perc_rmse,
        "Mejora_MAE_%": perc_mae
    })


    # Graficar y guardar
    plot_por_contaminante(
        target_col=target_col,
        train=train,
        test=test,
        forecast=forecast,
        y_true=y_true,
        y_pred_prophet=y_pred_prophet,
        y_pred_base=y_pred_base,
        h=h,
        modelo_prophet=m
    )

results_df = pd.DataFrame(all_results)
print(results_df)
results_df.to_csv("figs/resultados_metricas.csv", index=False)


DEBUG:cmdstanpy:input tempfile: /tmp/tmp2qdrgcbc/xojfqlim.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2qdrgcbc/f_a9cchm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=2635', 'data', 'file=/tmp/tmp2qdrgcbc/xojfqlim.json', 'init=/tmp/tmp2qdrgcbc/f_a9cchm.json', 'output', 'file=/tmp/tmp2qdrgcbc/prophet_model82fu033_/prophet_model-20250905042502.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
04:25:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
04:25:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2qdrgcbc/igzke32m.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2qdrgcbc/v7t1axne.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

      Variable  RMSE_Prophet  MAE_Prophet  RMSE_Baseline  MAE_Baseline  \
0  PM2.5.ug.m3      5.346124     3.656879       5.406766      3.413750   
1       O3.ppb      4.206510     2.717534       6.093029      3.791667   
2      NO2.ppb      3.425180     2.208742       3.508620      2.100000   

      ΔRMSE      ΔMAE  Mejora_RMSE_%  Mejora_MAE_%  
0  0.060642 -0.243129       1.121596     -7.122042  
1  1.886519  1.074133      30.961917     28.328783  
2  0.083440 -0.108742       2.378154     -5.178209  

✅ Listo. Figuras y métricas guardadas en la carpeta 'figs/'.
